In [1]:

import sys
import time
import os
import math
import cv2
import matplotlib
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
from ToTimeSeries import ToTimeSeries as ts_main


In [2]:
def find_patch_distribution(data):
    dist = np.zeros((len(INTENSITY_LEVELS),))
    for intensity in INTENSITY_LEVELS:
        mask = np.where((data[:,:] == intensity))
        dist[intensity] = len(data[mask[0], mask[1]])
    return dist
def turn_patch_gaussian(data):
    patch_dist = find_patch_distribution(data)
    patch_dist_positives = patch_dist[patch_dist>0]
    sigma = np.std(patch_dist_positives)
    mean = np.mean(patch_dist_positives) 
    gaussian_dist = np.empty_like(patch_dist_positives)
    if sigma == 0:
        gaussian_dist.fill(1)
        width = 0
        center = [len(gaussian_dist)/2]
    else:
        for k,n in enumerate(patch_dist_positives):
            g_num = ( math.e**(-((n - mean)**2)/(2 * sigma**2))/(math.sqrt(2*math.pi*(sigma**2)))) 
            gaussian_dist[k] = g_num
        width = np.max(gaussian_dist) - np.min(gaussian_dist)
        center = np.where(np.max(gaussian_dist)==gaussian_dist)[0]
    return width, center , gaussian_dist, sigma, mean

def turn_frame_gaussian(frame: np.ndarray, kernel: tuple):
    width_img, average_img = np.empty_like(frame[:,:,1], dtype=float), np.empty_like(frame[:,:,1])
    x_start = 0
    y_start = 0
    x_max = len(frame)
    y_max = len(frame[0])
    while y_max > y_start:
        while x_max > x_start:
            patch = frame[x_start: x_start+kernel[0], y_start: y_start+kernel[0]]
            if (y_start+kernel[1] >= y_max) or (x_start+kernel[0] >= x_max):
                pass
            start_time = time.time()
            width, center, gaussian_dist, sigma, mean = turn_patch_gaussian(patch[:,:, 0])
            width_img[x_start: x_start+kernel[1], y_start: y_start+kernel[0]] = width
            average_img[x_start: x_start+kernel[1], y_start: y_start+kernel[0]] = center[0]
            x_start+=kernel[0]
        x_start = 0
        y_start+= kernel[1]
    y_start=0
    return width_img, average_img
    
    
def manipulate_frame(frame, width_first_img):
    
    # width_first_img, average_img = turn_frame_gaussian(first_frame,(4,4))
    width_frame_img, average_frame_img = turn_frame_gaussian(frame, (4,4))
    background_mask = np.where(width_frame_img <= np.max(width_first_img)/3)
    platelet_mask = np.where(width_frame_img > np.max(width_first_img)/3)
    trinary_mask = np.where(frame[:,:,0] <= 120)
    quadric_mask = np.where(frame[:,:,0] <= 70)
    pantagonic_mask = np.where(frame[:,:,0] <= 40)
    frame[background_mask] = [255,255,255]
    frame[platelet_mask] = [0,0,0]
    frame[trinary_mask] = [255/2,255/2,255/2]
    frame[quadric_mask] = [255/3,255/3,255/3]
    frame[quadric_mask] = [255/4,255/4,255/4]
    # cv2.imwrite('test.png',last_frame)
    return frame
# cv2.imshow('test', last_frame)
# 
# cv2.waitKey(0) # waits until a key is pressed
# cv2.destroyAllWindows()
# exit()# destroys the window showing image

In [ ]:
INTENSITY_LEVELS = np.arange(0,256,1)
file_name = 't1.avi'
cap = cv2.VideoCapture('../ForVideoAnalyze/{0}'.format(file_name))
final_frame_count = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps = cap.get(cv2.CAP_PROP_FPS)

out = cv2.VideoWriter('../ForVideoAnalyze/results/IRM_Reasearch/{0}_manipulated.avi'.format(file_name),cv2.VideoWriter_fourcc('M','J','P','G'), fps, (frame_width, frame_height))
frame = None
first =True
ret = True
first_frame= None
width_img = None
temp = None
frame_ctr = 0
kernel = (4,4)
print("starting file:{0}, total frames:{1}".format(file_name, final_frame_count))
while ret:
    begin_timer = time.time()
    if first:
        ret, first_frame = cap.read()
        width_img, average_img = turn_frame_gaussian(first_frame, kernel)
        first_frame = manipulate_frame(first_frame, width_img)
        first = False
        out.write(first_frame)
        del first_frame
    else:
        frame = temp
        ret, temp =cap.read()
        if ret:
            frame = manipulate_frame(temp, width_img)
            out.write(frame)
    del frame
    print("finished frame number: {0} calc time ={1:.2f}".format(frame_ctr, time.time() - begin_timer))
    frame_ctr+=1
cap.release()
out.release()